## Домашнее задание 5
Голосная Мария, НПМбд-01-19

In [133]:
import numpy as np
import scipy.linalg

## LU-разложение
Матрицу А можно представить в виде произведения нижнетреугольной матрицы L с единичной диагональю и верхнетреугольной матрицы U.

In [134]:
def LU(a):
    a = np.array(a)
    n = len(a)
    l = np.eye(n)
    u = np.zeros((n, n))
    u[0, :] = a[0, :]
    l[:, 0] = a[:, 0] / u[0][0]
    for i in range(1, n):
        u[i, i] = a[i, i] - l[i, :] @ u[:, i]
        if i != n - 1:
            u[i, i + 1:] = a[i, i + 1] - l[i, :] @ u[:, i + 1]
            l[i + 1:, i] = (a[i + 1, i] - l[i + 1, :] @ u[:, i]) / u[i, i]
    return l, u

Решение СЛУ Аx=b сводится к решению следующих систем: Ly=b, Ux=y.

In [135]:
def solve_LU(a, b):
    a = np.array(a)
    b = np.array(b)
    n = len(a)
    l, u = LU(a)
    y = np.zeros(n)
    x = np.zeros(n)
    y[0] = b[0]
    for i in range(1, n):
        y[i] = b[i] - l[i, :i] @ y[:i]
    x[-1] = y[-1] / u[-1, -1]
    for j in range(n - 1, -1, -1):
        x[j] = (y[j] - u[j, j + 1:] @ x[j + 1:]) / u[j, j]
    return x

## LUP-разложение
Матрицу А можно представить в виде произведения PА=LU, где L - нижнетреугольная матрица с единичной диагональю, U - верхнетреугольная матрица, P - матрица перестановок, полученная из единичной матрицы.

In [136]:
def LUP(a):
    a = np.array(a, dtype=float)
    n = len(a)
    u = np.copy(a)
    l = np.zeros((n, n))
    p = np.eye(n)
    for i in range(n):
        m = np.argmax(abs(u[i:, i])) + i
        if m != i:
            u[i], u[m] = u[m], np.copy(u[i])
            l[i], l[m] = l[m], np.copy(l[i])
            p[i], p[m] = p[m], np.copy(p[i])
        l[i + 1:, i] = u[i + 1:, i] / u[i, i]
        u[i + 1:] -= l[i + 1:, i].reshape((n - i - 1, 1)) * u[i]
    l += np.eye(n)
    return l, u, p

Решение СЛУ Аx=b сводится к решению следующих систем: Ly=Pb, Ux=y.

In [138]:
def solve_LUP(a, b):
    a = np.array(a)
    b = np.array(b)
    n = len(a)
    l, u, p = LUP(a)
    y = np.zeros(n)
    x = np.zeros(n)
    y[0] = (p @ b)[0]
    for i in range(1, n):
        y[i] = (p @ b)[i] - l[i, :i] @ y[:i]
    x[-1] = y[-1] / u[-1, -1]
    for j in range(n - 1, -1, -1):
        x[j] = (y[j] - u[j, j + 1:] @ x[j + 1:]) / u[j, j]
    return x

## Проверка

In [141]:
a = [[4, 2, 5], [3, -2, 1], [7, 9, 3]]
b = [[2], [1], [3]]
solve_LU(a, b)

[0.31914894 0.04255319 0.12765957]


In [143]:
solve_LUP(a, b)

array([0.31914894, 0.04255319, 0.12765957])

In [142]:
scipy.linalg.solve(a, b)

array([[0.31914894],
       [0.04255319],
       [0.12765957]])